In [108]:
import requests
import json
from pathlib import Path
import os
import sys
import pandas as pd
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))

In [109]:
clientCrt = "../certificates/rw.crt"
clientKey = "../certificates/rw.key"

apiBaseUrl = "https://docucamrw.hesotech.eu/DocuCam/CameraOne/api/v1"
channelInfoUrl = apiBaseUrl + "/Data/ChannelInfo" 
imageApiUrl = apiBaseUrl + "/Data/ImageAndMeasurements"  # measurements are averages during scan time (scan time 5-30min)

headers = {'content-type': 'application/json'}

In [110]:
requestData = {
 
    # SiteName takes in two different parameters: [CameraOne, CameraTwo]
    # SceneName takes in different parameteres depending on SiteName:  [CameraOne [HospitalView]], [CameraTwo [SmallStreetSlide, FieldSlide, SchoolView]]

    # If you want to return all availalbe scans and images, comment out the Coordinates line.
   
    'SiteName': 'CameraTwo',
    'SceneName': 'SmallFieldSlide',
    'TimeRange': {
        'Minimum': '2023-01-01T00:00:00.000Z',
        'Maximum': '2023-02-01T00:00:00.000Z'
    },
    #'Coordinates': [ { 'Layer': 0, 'Row': 1, 'Col': 0 }, { 'Layer': 0, 'Row': 1, 'Col': 1 } ]  # comment out if you want to return all images
}

In [111]:
channelResponse = requests.get(channelInfoUrl, headers=headers, cert=(clientCrt, clientKey), allow_redirects=True)

if (channelResponse.status_code != 200):
    print('Something went wrong!')
    exit(-1)

In [112]:
response = requests.post(
    imageApiUrl,
    headers=headers,
    cert=(clientCrt, clientKey),
    allow_redirects=True,
    data=json.dumps(requestData)
)

if (response.status_code != 200):
    print('Something went wrong!')
    exit(-1)


In [121]:
channelInfoData = channelResponse.json()
data = response.json()
data
# for record in data:
#     print(record['Timestamp'], ' -> ', record['Values'], ' -> ', record['ImageUrls'])

[{'Timestamp': '2023-01-01T08:08:45.293837+00:00',
  'Values': [17.700000762939453,
   57,
   100000,
   718.2999877929688,
   5953,
   17.899999618530273,
   4.972222328186035,
   0,
   0,
   127,
   2],
  'ImageUrls': ['https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/0/1/0/2023-01-01T08-08Z',
   'https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/0/1/1/2023-01-01T08-08Z',
   'https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/1/2/1/2023-01-01T08-08Z',
   'https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/2/6/3/2023-01-01T08-08Z',
   'https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/2/6/4/2023-01-01T08-08Z',
   'https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/2/7/3/2023-01-01T08-08Z',
   'https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/2/7/4/2023-01-01T08-08Z',
   'https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/2/8/3/2023-01-01T08-08Z',
   'https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/2/8/4/2023-01-01T08-08Z']},
 {'Tim

In [133]:
# obtain measurement labels
skip_rate = 2
measurement_labels = []
for item in channelInfoData:
    for index, (key, value) in enumerate(item.items()):
        if index % skip_rate == 0:
            measurement_labels.append(value)

# map labels with measurements and drop redundant columns
df = pd.DataFrame(data)
count = 0
for i in measurement_labels:
    df[i] =df['Values'].apply(lambda x: x[count] )
    count = count + 1
df = df.drop(columns=['Values'])

In [134]:
df

,Timestamp,ImageUrls,ACTUAL_TEMPERATURE,HUMIDITY,ILLUMINATION,RAIN_COUNTER,SUNSHINEDURATION,WIND_SPEED,WIND_SPEED_CALC,Rain.Day.Counter,Rain.Diff,Sunshine.Day.Counter,Sunshine.Diff
0,2023-01-01T08:08:45.293837+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,17.700001,57,100000.0,718.299988,5953,17.900000,4.972222,0.0,0,127,2
1,2023-01-01T13:08:44.759627+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,18.700001,56,30490.0,718.299988,6253,24.900000,6.916667,0.0,0,427,3
2,2023-01-01T17:08:44.532268+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,15.900000,68,0.0,718.299988,6351,29.799999,8.277778,0.0,0,525,0
3,2023-01-02T08:08:45.427545+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,18.400000,52,65830.0,718.299988,6587,37.700001,10.472222,0.0,0,129,2
4,2023-01-02T13:08:44.942168+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.700001,56,34110.0,718.299988,6886,28.799999,8.000000,0.0,0,428,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2023-01-28T08:08:46.33017+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.600000,44,16180.0,773.200012,5921,25.700001,7.138889,0.0,0,130,2
75,2023-01-28T13:08:44.747983+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,20.500000,54,8380.0,773.200012,6221,23.400000,6.500000,0.0,0,430,3
76,2023-01-29T08:08:45.861822+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.299999,58,7448.0,773.200012,6524,16.900000,4.694445,0.0,0,128,2
77,2023-01-29T13:08:45.031074+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.600000,54,3942.0,773.200012,6825,13.300000,3.694444,0.0,0,429,2


# Questions
* How to return all images if timestamp is not known  --> like an "all" function...

In [ ]:
- 